**Цель исследования:**  


Проанализировать базу данных крупного сервиса для чтения книг по подписке.   

База содержит информацию о книгах, издательствах, авторах, а также пользовательские обзоры книг.  
Эти данные помогут сформулировать ценностное предложение для нового продукта.

---

In [13]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

**Таблица `books` :**

In [14]:
# функия для вывода результата запроса SQL
def sql_output(query):
    con=engine.connect()
    return pd.io.sql.read_sql(sql=text(query), con = con)

# текст SQL запроса
query_books = '''SELECT * FROM books LIMIT 5'''

sql_output(query_books)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


**Таблица `authors` :**

In [15]:
query_authors = '''SELECT * FROM authors LIMIT 5'''
sql_output(query_authors)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


**Таблица `publishers` :**

In [16]:
query_publishers = '''SELECT * FROM publishers LIMIT 5'''
sql_output(query_publishers)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


**Таблица `ratings` :**

In [17]:
query_ratings = '''SELECT * FROM ratings LIMIT 5'''
sql_output(query_ratings)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


**Таблица `reviews` :**

In [18]:
query_reviews = '''SELECT * FROM reviews LIMIT 5'''
sql_output(query_reviews)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Количество книг вышедших после 1 января 2000 года

In [19]:
query_1 = \
    ''' 
    SELECT COUNT(*)
    FROM books
    WHERE publication_date > '2000-01-01'
    '''
sql_output(query_1)

,count
0,819


Всего в базе данных 1000 книг из них **819 книг** было опубликовано с 2000 года.

---

### Количество обзоров и средняя оценка для каждой книги

In [24]:
query_2 = \
    ''' 
    WITH avg_rating AS 
    (SELECT book_id,
           ROUND(AVG(rating), 2) as book_avg_rating
    FROM ratings
    GROUP BY book_id),
    
    reviews_count AS 
    (SELECT book_id,
           COUNT(reviews) as book_reviews_count
    FROM reviews
    GROUP BY book_id)
    
    
    
    
    SELECT title,
           book_avg_rating,
           book_reviews_count
    FROM books 
    LEFT JOIN avg_rating ON books.book_id = avg_rating.book_id
    LEFT JOIN reviews_count ON books.book_id = reviews_count.book_id
    '''
sql_output(query_2)

,title,book_avg_rating,book_reviews_count
0,The Body in the Library (Miss Marple #3),4.50,2.0
1,Galápagos,4.50,2.0
2,A Tree Grows in Brooklyn,4.25,5.0
3,Undaunted Courage: The Pioneering First Missio...,4.00,2.0
4,The Prophet,4.29,4.0
...,...,...,...
995,The Cat in the Hat and Other Dr. Seuss Favorites,5.00,NaN
996,Anne Rice's The Vampire Lestat: A Graphic Novel,3.67,NaN
997,Essential Tales and Poems,4.00,NaN
998,Leonardo's Notebooks,4.00,NaN


У всех книг есть средняя оценка, но у 6 книг из 1000 отсутствуют обзоры.

---

### Издательства-лидеры по количеству выпущенных книг толще 50 страниц 

In [21]:
query_3 = \
    ''' 
    SELECT publisher,
           COUNT(book_id) book_published
    FROM books 
    JOIN publishers ON publishers.publisher_id = books.publisher_id
    WHERE num_pages > 50
    GROUP BY publisher
    ORDER BY book_published DESC
    LIMIT 5
    '''
sql_output(query_3)

,publisher,book_published
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


Наибольшее количество книг выпустило издательство **Penguin Books** - 42 книги.	

---

### Авторы с самой высокой средней оценкой книг (книги с 50 и более оценками)

In [22]:
 query_4 = \
    ''' 
    WITH r AS 
    (SELECT book_id,
            COUNT(rating) rating_count,
            AVG(rating) avg_rating
    FROM ratings
    GROUP BY book_id),
    
      
    ar AS
    (SELECT title,
           author,
           rating_count,
           avg_rating
    FROM books as b
    JOIN r ON b.book_id = r.book_id
    JOIN authors as a ON a.author_id = b.author_id
    )
    
    SELECT author,
           ROUND(AVG(avg_rating), 2) author_avg_rating
    FROM ar
    WHERE rating_count >= 50
    GROUP BY author
    ORDER BY author_avg_rating DESC
    LIMIT 10
    '''
sql_output(query_4)

,author,author_avg_rating
0,J.K. Rowling/Mary GrandPré,4.28
1,Markus Zusak/Cao Xuân Việt Khương,4.26
2,J.R.R. Tolkien,4.26
3,Louisa May Alcott,4.19
4,Rick Riordan,4.08
5,William Golding,3.90
6,J.D. Salinger,3.83
7,William Shakespeare/Paul Werstine/Barbara A. M...,3.79
8,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.79
9,Lois Lowry,3.75


Самый высокий средний рейтинг у **Дж.К. Роулинг** / Мэри Гранпрэ (иллюстратор книг)

---


### Активные пользователи: среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [23]:
query_5 = \
    '''
    SELECT COUNT(review_id) / COUNT(DISTINCT username)
    FROM reviews
    WHERE username IN
    (SELECT username
    FROM ratings
    GROUP BY username
    HAVING  COUNT(rating) > 48)
    '''
sql_output(query_5)

,?column?
0,24


Из всех пользователей только 13 поставили более 48 оценок книгам. На этих активных пользователей приходится 312 обзоров, т.е. в **среднем 24 обзора на пользователя** или другими словами - **активный пользователь на каждые две поставленные оценки делает один обзор**.

---

### Итоговый вывод:

- Всего в базе данных 1000 книг из них 819 книг было опубликовано с 2000 года.


- Наибольшее количество книг выпустило издательство Penguin Books - 42 книги.


- Самый высокий средний рейтинг из авторов у Дж.К. Роулинг / Мэри Гранпрэ (иллюстратор книг) - 4.28 (мах - 5).


- Из всех пользователей только 13 поставили более 48 оценок книгам. На этих активных пользователей приходится 312 обзоров, т.е. в среднем 24 обзора на пользователя или другими словами - активный пользователь на каждые две поставленные оценки делает один обзор.